In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import anndata as ad
import scanpy as sc
import os
import squidpy as sq
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, accuracy_score
import xgboost as xgb
import catboost
import json


from tqdm import tqdm
import sys
sys.path.append('/Users/rssantanu/Desktop/codebase/constrained_FM')


import argparse

max_rows = 1000000
pd.set_option('display.max_rows', max_rows)
pd.set_option('display.max_columns', max_rows)
pd.set_option('display.width', max_rows)


/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)


In [2]:
scRNA = ad.read_h5ad(os.path.join('/Users/rssantanu/Desktop/codebase/constrained_FM', 'datasets/h5ad_processed_datasets', 'GSE232025_stereoseq.h5ad'))


/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [3]:
scRNA.obs['day'].value_counts()

day
3    11698
4     8243
2     4410
1     2929
0     1477
Name: count, dtype: int64

In [4]:
import liana as li

In [5]:
import re

def extract_symbol(name):
    # Try to find a gene symbol in the string
    # Look for patterns like 'SYMBOL[nr]' or 'SYMBOL[hs]' or 'SYMBOL |'
    match = re.search(r'([A-Z0-9\-]+)(?:\[.*?\])?\s*\|', name)
    if match:
        return match.group(1)
    # If not found, try to find a symbol before a space or bracket
    match = re.match(r'([A-Z0-9\-]+)', name)
    if match:
        return match.group(1)
    return None

# Apply to your var_names
symbols = [extract_symbol(x) for x in scRNA.var_names]
scRNA.var['gene_symbol'] = symbols
# Filter out None or nan
scRNA = scRNA[:, [s is not None and s != 'nan' for s in symbols]]
scRNA.var_names = scRNA.var['gene_symbol']

/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [6]:
# for all the days

li.method.rank_aggregate(
    adata=scRNA,
    groupby='celltype',
    resource_name='consensus',
    use_raw=False,         # <--- add this line
    verbose=True
)

scRNA_by_day= []

for stage in scRNA.obs['day'].unique():
    adata_stage = scRNA[scRNA.obs['day'] == stage].copy()
    li.method.rank_aggregate(
        adata=adata_stage,
        groupby='celltype',
        resource_name='consensus',
        use_raw=False,
        verbose=True
    )
    # Save or analyze results for this stage
    # adata_stage.uns['liana_res'].to_csv(f'liana_results_stage_{stage}.csv')
    scRNA_by_day.append(adata_stage)



Using resource `consensus`.
Using `.X`!
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:381: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable nam

Generating ligand-receptor stats for 28757 samples and 119 features
Assuming that counts were `natural` log-normalized!


/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/liana/method/sc/_liana_pipe.py:262: ImplicitModificationWarning: Setting element `.layers['scaled']` of view, initializing view as actual.
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


Running CellPhoneDB


100%|██████████| 1000/1000 [00:06<00:00, 151.05it/s]


...

Generating ligand-receptor stats for 8243 samples and 119 features
Assuming that counts were `natural` log-normalized!


/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/liana/method/sc/_liana_pipe.py:262: ImplicitModificationWarning: Setting element `.layers['scaled']` of view, initializing view as actual.


Running CellPhoneDB


100%|██████████| 1000/1000 [00:02<00:00, 447.10it/s]


Running Connectome
Running log2FC
Running NATMI
Running SingleCellSignalR


In [7]:
liana_df= scRNA.uns['liana_res']

In [29]:
liana_df[(liana_df['source']=='Immature dpEX') & (liana_df['target']=='Immature dpEX')]

,source,target,ligand_complex,receptor_complex,lr_means,cellphone_pvals,expr_prod,scaled_weight,lr_logfc,spec_weight,lrscore,specificity_rank,magnitude_rank
1719,Immature dpEX,Immature dpEX,NPTX2,NPTXR,2.364933,0.000,5.091072,1.534250,1.539298,0.008971,0.934585,0.001802,0.000009
1713,Immature dpEX,Immature dpEX,ADCYAP1,ADCYAP1R1,1.725900,0.000,2.941888,1.694290,1.528534,0.009456,0.915687,0.001802,0.002703
1716,Immature dpEX,Immature dpEX,COL18A1,GPC4,0.141312,0.951,0.017259,-0.075883,-0.232093,0.000871,0.454103,1.000000,0.440814
1723,Immature dpEX,Immature dpEX,SFRP1,FZD2,0.139863,1.000,0.019099,-0.123248,-0.376602,0.000480,0.466685,1.000000,0.538556
1725,Immature dpEX,Immature dpEX,SPARC,ENG,0.437105,0.000,0.130964,0.257370,0.065773,0.003688,0.696185,0.263333,0.539938
1726,Immature dpEX,Immature dpEX,SPARC,FGFR1,0.405186,0.004,0.087411,0.089180,-0.068230,0.001478,0.651819,0.626644,0.540341
1715,Immature dpEX,Immature dpEX,COL18A1,GPC1,0.788997,0.000,0.132873,0.969895,0.725866,0.003139,0.697713,0.030461,0.550902
1718,Immature dpEX,Immature dpEX,L1CAM,CD9,0.916338,0.000,0.211199,0.494056,0.377694,0.001533,0.744242,0.266269,0.624284
1724,Immature dpEX,Immature dpEX,SLIT2,GPC1,0.800391,0.000,0.166800,1.027187,0.782163,0.003848,0.721142,0.030461,0.785748
1714,Immature dpEX,Immature dpEX,BCAN,NRCAM,0.746677,0.000,0.167230,0.528328,0.410530,0.001530,0.721400,0.269824,0.787822


In [9]:
# import liana as li

# li.pl.dotplot(
#     adata=scRNA,
#     colour='lr_means',           # or another score column
#     size='specificity_rank',     # or another ranking/stat column
#     inverse_size=True,           # small p-values = big dots
#     source_labels=['Immature dpEX', 'Immature mpEX'],  # replace with your cell types
#     target_labels=['Immature nptxEX', 'Immature mpEX'],  # replace with your cell types
#     figure_size=(9, 5),
#     filter_fun=lambda x: x['cellphone_pvals'] <= 0.05,  # filter by significance
#     cmap='plasma'
# )

In [10]:
# scRNA_by_day[2].uns['liana_res']

In [11]:
# for scrna_day in scRNA_by_day[2]:

#     li.pl.dotplot(
#         adata=scrna_day,
#         colour='lr_means',           # or another score column
#         size='specificity_rank',     # or another ranking/stat column
#         inverse_size=True,           # small p-values = big dots
#         source_labels=['Immature dpEX', 'Immature mpEX'],  # replace with your cell types
#         target_labels=['Immature nptxEX', 'Immature mpEX'],  # replace with your cell types
#         figure_size=(9, 5),
#         filter_fun=lambda x: x['cellphone_pvals'] <= 0.05,  # filter by significance
#         cmap='plasma'
#     )

In [41]:
def get_communication_matrix(adata, all_celltypes):

    communication_dict = {}
    liana_df = adata.uns['liana_res']
    for celltype in all_celltypes:
        communication_dict[celltype] = []
        for celltype_2 in all_celltypes:
            if celltype in liana_df['source'].unique() and celltype_2 in liana_df['target'].unique():
                communication_dict[celltype].append(liana_df[(liana_df['source'] == celltype) & (liana_df['target'] == celltype_2)]['lrscore'].values.sum())
            else:
                communication_dict[celltype].append(0)

    cc_comm_df= pd.DataFrame(communication_dict, index=all_celltypes, columns=all_celltypes)

    return cc_comm_df

In [45]:
all_celltypes = scRNA.obs['celltype'].unique()

In [46]:
all_data_comm= get_communication_matrix(scRNA, all_celltypes)

In [47]:
data_comm_0 = get_communication_matrix(scRNA_by_day[0], all_celltypes)
data_comm_1 = get_communication_matrix(scRNA_by_day[1], all_celltypes)
data_comm_2 = get_communication_matrix(scRNA_by_day[2], all_celltypes)
data_comm_3 = get_communication_matrix(scRNA_by_day[3], all_celltypes)
data_comm_4 = get_communication_matrix(scRNA_by_day[4], all_celltypes)








In [49]:
data_comm_4

,Immature nptxEX,VLMC,CP,dNBL1,Immature MSN,dNBL2,Immature CMPN,dEGC,dNBL3,Immature cckIN,Immature dpEX,dNBL4,Immature DMIN,wntEGC,sfrpEGC,Immature mpEX,dNBL5,nptxEX,MSN,sstIN,Unknown,tlNBL,dpEX,MCG,CMPN,npyIN,cckIN,scgnIN,ribEGC,mpEX,ntng1IN,mpIN
Immature nptxEX,0,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
VLMC,0,4.111259,3.621933,0,0,0,0,0,0,0,0,0,0,9.000273,8.895769,0,0,5.554606,4.949380,4.702588,0,6.463320,4.226785,0,4.979526,0,4.906732,5.136333,4.505177,4.155336,5.964667,4.177746
CP,0,2.102047,1.611226,0,0,0,0,0,0,0,0,0,0,4.979166,4.937141,0,0,3.723115,3.110575,2.879082,0,4.518277,2.332543,0,2.602777,0,3.119812,2.703276,1.769893,2.304402,3.621109,2.343680
dNBL1,0,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Immature MSN,0,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
dNBL2,0,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Immature CMPN,0,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
dEGC,0,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
dNBL3,0,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Immature cckIN,0,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
